In [85]:
import pandas as pd
import requests

In [59]:
api_url = "https://api.pokemontcg.io/v2/cards?q=set.id:sv3pt5&pageSize=207&select=id,name,number,rarity,tcgplayer"
api_key = ''

headers = {
    'X-Api-Key': api_key
}

json_response = None

try:
    response = requests.get(api_url, headers=headers)
    response.raise_for_status()
    json_response = response.json()
except requests.RequestException as e:
    print(f"Request failed: {e}")

In [97]:
tcgplayer_df = pd.json_normalize(json_response['data'], sep='_')

col = [
 'tcgplayer_prices_reverseHolofoil_market',
 'tcgplayer_prices_normal_market',
 'tcgplayer_prices_holofoil_market']

tcgplayer_df["MP"] = tcgplayer_df[col].median(axis=1,skipna=True)
tcgplayer_df["Market Price"] = tcgplayer_df["MP"].apply(lambda x: "${:.2f}".format(x))

In [98]:
tcgplayer_df = tcgplayer_df.rename(columns={"number":"ID","name":"Card Name","rarity":"Rarity","tcgplayer_url":"TCG Player URL"})
tcgplayer_df["ID"] = tcgplayer_df["ID"].astype('int64')

In [88]:
try:
    path = "C:\\Program Files\\PokemonInventory\\application\\Pokemon151Inventory.xlsx"
    path = "C:\\Users\\nicho\\Downloads\\Pokemon\\Pokemon151Inventory.xlsx"
    inventory_df = pd.read_excel(path)
except FileNotFoundError as e:
    print("No file")
except Exception as e:
    print(e)

In [99]:
main_df = pd.merge(tcgplayer_df, inventory_df, how="left", on='ID')
outstanding_df = main_df[main_df['Count'] < 1]
owned_df = main_df[main_df['Count'] > 0]

breakdown_atr = ['ID','Card Name','Rarity',"Market Price"]
outstanding_cost_breakdown_df = outstanding_df[breakdown_atr]
owned_cost_breakdown_df = owned_df[breakdown_atr]

In [104]:
n_outstanding = outstanding_df["ID"].count()
oustanding_MP = "${:.2f}".format(outstanding_df["MP"].sum())
n_owned = owned_df["ID"].count()
owned_MP = "${:.2f}".format(owned_df["MP"].sum())